# RAG с текстовым поиском на основе Yandex Search API

В данном примере рассмотрим возможность построения RAG с использованием текстового поиска на основе [Yandex Search API](https://yandex.cloud/ru/services/search-api)  

Для данного примера необходимо:
- **GigaChat API**: получить [авторизационные данные](https://developers.sber.ru/docs/ru/gigachain/get-started/quickstart#nastroyka-avtorizatsii-zaprosov-k-giga-chat-api)
- **Yandex Search API**: получить идентификатор каталога (folderid) и API-ключ (apikey) для [аутентификации в сервисе](https://yandex.cloud/ru/docs/search-api/operations/auth)

In [15]:
from getpass import getpass
import os

from dotenv import find_dotenv, load_dotenv

In [2]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass(f"{key}:")

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
_set_env("GIGACHAT_CREDENTIALS")
_set_env("YANDEX_API_KEY")
_set_env("YANDEX_FOLDER_ID")

## Конфигурация

In [5]:
from operator import itemgetter
from textwrap import dedent

from IPython.display import Markdown
from langchain_gigachat.chat_models import GigaChat
from langchain_community.retrievers.yandex_search import YandexSearchAPIRetriever
from langchain_community.utilities.yandex_search import YandexSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel

Определим дополнительные функции для форматирования документов до отправки их в LLM (`format_docs`) и для формирования финального ответа (`generate_final_answer`).

In [6]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


def generate_final_answer(response):
    final_answer_template = dedent(
        """
        ### Вопрос
        {question}
        
        ### Ответ
        {answer}
        
        ### Источники
        {sources}"""
    ).strip()

    final_answer = final_answer_template.format(
        question=response["question"],
        answer=response["answer"],
        sources="\n\n".join(
            f'{doc.page_content} ({doc.metadata["url"]})' for doc in response["context"]
        ),
    )

    return final_answer

Используем модель `GigaChat-Pro` для данной задачи.

In [7]:
model = GigaChat(
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

Создадим клиента для Yandex Search API с параметрами по умолчанию.  
Детальную информацию по дополнительным параметрам можно уточнить в [документации](https://yandex.cloud/ru/docs/search-api/concepts/get-request).

In [9]:
api_wrapper = YandexSearchAPIWrapper()

Настроим retriever на основе созданного клиента, получая первые k наиболее релевантных ответов на запрос.  
В случае, если клиент для Yandex Search API используется с параметрами по умолчанию, то можно не передавать аргумент api_wrapper для retriever.

In [10]:
retriever = YandexSearchAPIRetriever(api_wrapper=api_wrapper, k=30)

Создадим chain для генерации ответа от LLM с возможностью получения информации по документам, которые использовались при формировании ответа.

In [11]:
QA_TEMPLATE = """Ответь на следующий вопрос на основе предоставленного контекста:

{context}

Вопрос: {question}
Ответ: 
"""
prompt = ChatPromptTemplate.from_template(QA_TEMPLATE)

output_parser = StrOutputParser()

chain_without_source = (
    RunnableParallel(
        {
            "context": itemgetter("context") | RunnableLambda(format_docs),
            "question": itemgetter("question"),
        }
    )
    | prompt
    | model
    | output_parser
)
chain_with_source = RunnableParallel(
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
).assign(answer=chain_without_source)

## Примеры использования

In [12]:
query = "Какие концерты проходят в Москве в августе 2024 года?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Какие концерты проходят в Москве в августе 2024 года?

### Ответ
В августе 2024 года в Москве запланировано множество концертов различных жанров и исполнителей. Среди них концерт оркестра Sonorus "Гарри Поттер", выступления как известных, так и начинающих музыкантов, концерты классической музыки от благотворительного фонда "Бельканто", рок-концерты, джазовые выступления, сольные концерты Юрия Каспаряна и группы "Звери". Также в августе пройдут фестивали, включая "Пикник Афиши" и "Лето в Москве".

### Источники
Концерт оркестра Sonorus «Гарри Поттер» 2 августа 2024, 19:00 Дом кинематографистов.
Здесь собраны лучшие концерты Москвы по рейтингу и нашим рекомендациям. Практический каждый день в столице проходят яркие концерты и шоу. Выступают как уже состоявшие артисты, так и только начинающие, но уже заявившие о себе музыканты. (https://www.kp.ru/afisha/msk/konczerty/luchshie-kontserty-moskvy/avgust/)

Купить билеты на Концерты в Москве в августе 2024 на Яндекс Афише. (https://afisha.yandex.ru/moscow/concert/aug)

На этой странице перечислены концерты в Москве. Афиша и расписание выступлений артистов в августе 2024 года которые проходят сегодня, либо будут проходить скоро: например завтра, на этих выходных и т.д. Кудамоскоу в курсе 12 музыкальных событий и концертов, которые пройдут в Москве в августе 2024 года. (https://kudamoscow.ru/event/concert/08-2024/)

Купить билеты на концерт в Москве на август 2024, афиша концертов и цены на них. (https://www.afisha.ru/msk/schedule_concert/na-avgust/)

В случае, если третье лицо (правообладатель или уполномоченное им лицо) считает, что его права на объект интеллектуальной собственности нарушаются, он может направить претензию по адресу: 109147, г. Москва, вн.тер.г. муниципальный округ Таганский, ул. Воронцовская, д. 1/3, стр. 2А и по e-mail: support@ticketland.ru.
Концерты в августе 2024 года. Найдено 825 мероприятий. (https://www.ticketland.ru/concert/2024/8/)

Билеты на лучшие концерты в Москве в Июле 2024 года. Успейте купить заранее по низким ценам! (https://concert.moscow/august-2024/)

Благотворительный фонд «Бельканто» в Москве предлагает Вам посетить концерты классической музыки в августе 2024 года. Здесь вы найдете полную афишу классических концертов в Москве на август 2024 года: большой репертуар, онлайн продажа билетов, удобный поиск и подробное описание концертов. (https://www.belcantofund.com/calendar/2024/08/)

Билеты на концерты в Москве 2024, полное расписание выступлений артистов на всех площадках города. (https://koncert-2024.ru/moscow/month-august)

03 августа. Сб. ДК Горбунова, Новозаводская ул., 27. (https://mosconcert.moscow/afisha/?month=8&year=2024)

Концерты в Москве в августе 2024 Расписание ближайших концертов, гастролей удобный календарь событий. Продажа билетов на концерты на MSK.KASSIR.RU, звоните! (https://msk.kassir.ru/bilety-na-koncert/avgust)

Концерты в Москве в августе 2024 года: афиша концертов на сайте «Concert Russia». (https://concert-russia.ru/moscow/month-august)

Ближайшие концерты в бард-клубе Гнездо глухаря, афиша... (https://gnezdogluharya.ru/concerts/2024-08)

Афиша концертов в Москве на август 2024 года, полное расписание выступлений в августе в Москве, купите билеты онлайн на официальном сайте. (https://afisha-concert.ru/moscow/list/august)

Афиша концертов в Москве август 2024. Низкие цены на билеты от официальных продавцов. (https://rolld.ru/msk/concert/august)

Афиша Москвы в августе 2024, куда сходить. 100 самых интересных событий: Духовные Принципы - МалоСольный концерт!, Спектакль «Ответ Гиппократа», Концерт Андрея Виноградова (hurdy-gurdy), Большой женский Stand-Up концерт, Константин Кондрашов и друзья! (https://msk.zeroevent.ru/2024-08)

Все рок-концерты в Москве. Узнай больше о расписании рок-концертов в августе 2024 года, местах и ценах на билеты на MyRockShows. (https://ru.myrockshows.com/location/166-moscow/aug-2024/)

Афиша Москвы на август 2024 года. Все мероприятия в августе в Москве: концерты, спектакли, спорт, для детей, фестивали, театр. (https://afisha-2024.ru/moscow/month-august)

Приближается последний месяц лета, в столице он традиционно насыщен культурными и развлекательными событиями. Если вы еще не спланировали досуг, то наша сегодняшняя подборка поможет в этом. Предлагаем ознакомиться с мероприятиями, которые будут проходить в Москве и Московской области в августе 2024 года. (https://dzen.ru/a/ZqAYi3J4tB92ABaW)

Лучшие концерты Москвы ждут Вас! Билеты на концерты, подробнейшая афиша, заказ и доставка по Москве и МО. Купить билеты можно сделав заказ по телефону 229-04-00 или онлайн. (https://biletservis.ru/concerts/august/)

Расписание концертов и гастролей на август 2023 года в Москве. (https://getafisha.ru/concert/august.php)

Концерт Юрия Каспаряна 16+. Один из самых ярких и влиятельных гитаристов России, известный по участию в группе «Кино», выступит в Москве со своим проектом YK.  14 августа 20:00.  Клуб «16 Тонн».
Москва-Сити, фотосессия и ужин от 4300р. Свидание для двоих на 67 этаже. Главное летом 2024. Лето в городе. (https://kudago.com/msk/list/summer-list-music/)

Самые ожидаемые события августа 2024 в Москве Рассказываем о том, чем заняться в августе в Москве: сходить на пикник «Афиши» в «Лужниках», заглянуть на маркет на Veter Fest, сделать шопинг на «Ламбада Маркет» и не только.
24 августа готовимся покорять «Лужники» на сольном концерте «Зверей», 21 августа джаз-клуб Игоря Бутмана на концерте Валерия Сюткина и ансамбля S.O.S.(plus), а 30 августа «Зеленый театр» на ВДНХ на концерте обворожительной Любови Успенской. (https://top15moscow.ru/novosti/samye-ozhidaemye-sobytiya-avgusta-2024-v-moskve)

Август: от фестивалей и шоу до конференций и саммитов - информация обо всех актуальных событиях месяца на gorodzovet.ru. (https://gorodzovet.ru/moskva/2024/august/)

Календарь концертов: август 2024 года. ROCKGIG: полная афиша московских концертов. (https://rockgig.ru/calendar/2024-08)

Август 2024 года - это месяц, когда в России проходит множество различных фестивалей, на которых можно провести время с пользой и удовольствием. Ниже представлены некоторые из них: Фестиваль "Пикник Афиши" - это одно из самых крупных музыкальных событий года, которое проходит в начале августа в Москве. (https://open-air.ru/afisha/festivali-v-avguste?city=%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F)

С начала лета и до 8 сентября более чем на 600 площадках Москвы (в парках, скверах, набережных, бульварах, на фестивальных площадках «Московских сезонов» и т. д.) будет проходить крупнейший городской фестиваль «Лето в Москве.
В саду «Эрмитаж» 10–11 августа пройдет Фестиваль Короткой Новой прозы «БеспринцЫпные чтения 2024» (16+). (https://msk1.ru/text/entertainment/2024/06/09/73679507/)

Афиша концертов Москвы. Билеты на концерты без наценки и с кешбэком 10% для всех! Попробуйте МТС Лайв и получите кешбэк 20% за первую покупку. (https://live.mts.ru/moscow/concerts)

Все рок-концерты в России 2024. (https://www.rockafisha.com/ru/2024/08/)

Афиша концертов в Москве в августе 2024 года. Концертная афиша Концертные залы и клубы Фестивали Рейтинги Статьи. (https://www.vashdosug.ru/msk/concert/schedule/august/)

Великолепные концерты на 2024 год: билеты онлайн, полное расписание и концертные площадки в Москве. (https://redevents.ru/events)

In [13]:
query = "Как прошел финал Лиги чемпионов в 2024 году?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Как прошел финал Лиги чемпионов в 2024 году?

### Ответ
Финал Лиги чемпионов в 2024 году прошёл 1 июня на стадионе "Уэмбли" в Лондоне. В матче встретились дортмундская "Боруссия" и мадридский "Реал". Испанский клуб одержал победу со счетом 2:0 благодаря голам Даниэля Карвахаля и Винисиуса Жуниора. Мадридский "Реал" в 15-й раз в истории стал победителем Лиги чемпионов.

### Источники
Финал Лиги чемпионов УЕФА 2024 года прошёл 1 июня 2024 года на стадионе «Уэмбли» в Лондоне. Этот футбольный матч стал 69-м финалом Лиги чемпионов УЕФА... (https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_%D0%9B%D0%B8%D0%B3%D0%B8_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2024)

Мадридский «Реал» стал победителем Лиги чемпионов УЕФА, обыграв в финале дортмундскую «Боруссию» со счетом 2:0. Матч проходил на стадионе «Уэмбли» в Лондоне.
Кроос выступал за клуб с 2014 года, вместе с «Реалом» он пять раз выигрывал Лигу чемпионов (еще один раз — с «Баварией»). Также Кроос выступал за сборную Германии, вместе с которой выиграл чемпионат мира 2014 года. В мае он объявил, что завершит карьеру после Евро-2024. (https://meduza.io/news/2024/06/02/real-vyigral-ligu-chempionov)

После трансформации Кубка европейских чемпионов в Лигу чемпионов (1991) на именитом стадион прошел третий финал (2011, 2013).
64-летний Карло Анчелотти выиграл свою пятую Лигу чемпионов в качестве тренера — трижды с «Реалом» (2014, 2022, 2024) и дважды с итальянским «Миланом» (2003, 2007). (https://www.rbc.ru/sport/02/06/2024/665b27549a79473af88e70a7)

Завершился финал Лиги чемпионов УЕФА, в котором встречались дортмундская «Боруссия» и мадридский «Реал».
Предыдущим победителем Лиги чемпионов является «Манчестер Сити», обыгравший в финале сезона-2022/2023 миланский «Интер» со счётом 1:0. Единственный гол в матче забил полузащитник «горожан» Родри. (https://www.championat.com/football/news-5574730-borussiya-real-0-2-rezultat-finala-ligi-chempionov-sezona-2034-2034-1-iyunya-2024.html)

Решающий матч Лиги чемпионов УЕФА-2023/24 пройдет в Лондоне. (https://ru.uefa.com/uefachampionsleague/news/0282-1827072c4017-5a3b442ec8c7-1000--%D1%87%D1%82%D0%BE-%D0%BD%D0%B0%D0%B4%D0%BE/)

Рассказываем, кто выиграл Лигу Чемпионов в 2024 году и как проходил финал главного футбольного турнира.
Победа «Реала Мадрид» в 15-й раз в истории Лиги Чемпионов не только подчеркивает доминирующее положение клуба на европейской арене, но и продолжает традицию великих достижений, которыми так славится мадридский клуб. (https://spb.tsargrad.tv/news/kto-vyigral-ligu-chempionov-2024-kak-prohodil-final-glavnogo-futbolnogo-turnira_1008781)

Мадридский "Реал" обыграл "Боруссию" из Дортмунда в финале Лиги чемпионов УЕФА со счетом 2:0 и в 15-й раз завоевал главный клубный трофей Европы.
Для "Боруссии" из Дортмунда участие в финале Лиги чемпионов УЕФА стало первым с 2013 года, когда она проиграла соотечественникам из "Баварии" . (https://www.dw.com/ru/madridskij-real-stal-pobeditelem-ligi-cempionov-uefa/a-69243033)

Финал Лиги чемпионов 2024 года состоялся в субботу, 1 июня, на стадионе «Уэмбли» в Лондоне. В матче встретились дортмундская «Боруссия» и мадридский «Реал». Испанский клуб одержал победу со счетом 2:0 благодаря голам Даниэля Карвахаля на 74-й минуте и Винисиуса Жуниора на 83-й минуте. Мадридский «Реал» в 15-й раз в истории стал победителем Лиги чемпионов. (https://dzen.ru/a/ZluU0dA-jxLsd3EH)

Смотрите онлайн Обзор финального матча Лиги чемпионов 2023-2.. (https://vk.com/video-179087678_456244563)

Финал Лиги чемпионов в 2025 году пройдет в Мюнхене на «Альянц Арене». С сезона-2024/25 формат Лиги чемпионов претерпит изменения — число участников увеличится до 36, они будут играть в одной группе по швейцарской системе. Клубы проведут по восемь матчей против разных команд — по четыре дома и в гостях, пары определятся в результате жеребьевки после посева. (https://sportmail.ru/news/football-eurocups/61334157/)

«Боруссия» — «Реал»: финал Лиги чемпионов сезона 2023 / 2024 — дата и место проведения, стадион «Уэмбли» (Лондон), время начала и онлайн трансляция матча, результат и главные материалы — новости и обзор игры 1 июня. (https://www.sport-express.ru/football/champions-league/stories/borussiya-real-final-ligi-chempionov-2024-rezultat-i-glavnye-materialy-novosti-i-obzor-matcha-1-iyunya-2208494/)

Узнайте, как прошел финальный матч Лиги чемпионов 2023/24 «Боруссия» – «Реал» и кто стал победителем... (https://olympics.com/ru/news/futbol-liga-chempionov-2023-2024-madridskii-real-vyigral-turnir-v-15-i-raz)

2024 UEFA Champions League Final.
Этот финал Лиги чемпионов УЕФА стал третьим, проведённым на новом «Уэмбли» (после финалов 2011 и 2013 годов) и восьмым финалом турнира, проведённым в Лондоне (пять других финалов прошли ещё на старом «Уэмбли» — в 1963, 1968, 1971, 1978 и 1992 годах). (https://ru.ruwiki.ru/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_%D0%9B%D0%B8%D0%B3%D0%B8_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2024)

Мадридский "Реал" обыграл дортмундскую "Боруссию" в финале футбольной Лиги чемпионов.Встреча прошла в Лондоне на стадионе "Уэмбли" и завершилась победой "королевского клуба" со счетом 2:0. Голы забили Даниэль Карвахаль (74-я минута) и Винисиус Жуниор (83).
Мадридский "Реал" обыграл дортмундскую "Боруссию" в финале футбольной Лиги чемпионов. (https://rsport.ria.ru/20240601/match-1949944744.html)

Мадридский "Реал" стал победителем Лиги чемпионов. В финале испанский клуб со счетом 2:0 обыграл дортмундскую "Боруссию". (https://rg.ru/2024/06/02/madridskij-real-v-15-j-raz-vyigral-ligu-chempionov.html)

Завершился финал Лиги чемпионов УЕФА сезона 2023-2024, в котором встречались дортмундская "Боруссия" и мадридский "Реал". Команды сыграли на стадионе "Уэмбли" в Лондоне. Встреча завершилась победой "королевского клуба". На 74-й минуте счет в игре открыл защитник "Реала" Даниэль Карвахаль. А на 83-й минуте матча, преимущество мадридцев удвоил нападающий Винисиус Жуниор, установив итоговый счет встречи – 2:0. (https://smotrim.ru/article/3979922)

Мадридский "Реал" обыграл дортмундскую "Боруссию" со счетом 2:0 и стал победителем Лиги чемпионов. Финальный матч состоялся на стадионе "Уэмбли" в Лондоне. (https://ren.tv/news/sport/1226014-real-obygral-borussiiu-i-stal-pobeditelem-ligi-chempionov)

Мадридский «Реал» одержал победу над дортмундской «Боруссии» в финале Лиги чемпионов 2024 со счетом 2:0. Испанский клуб стал лучшим в Европе в 15-й раз. Игра финала Лиги чемпионов прошла на стадионе «Уэмбли» в Лондоне.
До 1 июня самым титулованным игроком в истории Лиги чемпионов был игрок «Реала» Пако Хенто. Он одержал шесть побед в финалах турнира, последнюю — в 1966 году, сообщает РБК. (https://vtomske.ru/news/204646-real-vyigral-final-ligi-chempionov-2024)

Мадридский «Реал» победил дортмундскую «Боруссию» в финале Лиги чемпионов сезона-2023/2024. Об этом сообщает корреспондент «Ленты.ру»... (https://lenta.ru/news/2024/06/01/real-v-15-y-raz-vyigral-ligu-chempionov/)

Что случилось в финале Лиги чемпионов «Боруссия» — «Реал».
«Реал» был безусловным фаворитом финала Лиги чемпионов. Самый богатый клуб мира недавно завоевал 36-й титул лучшей команды Испании, обогнав на 10 очков «Барселону». В ЛЧ мадридцы по пути к решающему матчу справились с грозным «Манчестер Сити». (https://www.kp.ru/sports/futbol/chto-sluchilos-v-finale-ligi-chempionov-borussiya-real-2024/)

Финал ЛЧ, прошедший первого июня на Уэмбли, завершился будничной победой мадридцев - 2:0. Голы забили заслуженный ветеран клуба и испанского футбола правый защитник Дани Карвахаль, и уже не такой молодой и перспективный, но находящийся на пике карьеры и формы 23-летний бразильский форвард Винисиус Джуниор. (https://www.sports.ru/tribuna/blogs/merlininews/3245395.html)

Мадридский «Реал» обыграл дортмундскую «Боруссию» в финале Лиги чемпионов.
При этом за 3 минуты до конца основного времени рефери не засчитал гол немецкой команды. Предыдущим победителем Лиги чемпионов был «Манчестер Сити», обыгравший в финале миланский «Интер» со счётом 1:0. «Реал» выиграл Лигу чемпионов в 15-й раз. (https://www.mk.ru/sport/2024/06/01/madridskiy-real-stal-pobeditelem-ligi-chempionov.html)

Финал Лиги чемпионов 2024 года Лига чемпионов.
В 1/8 финала мадридцы не без труда прошли «Лейпциг» (1:0 и 1:1), а вот дальше «Реал» снова доказал всем, что Лига чемпионов его турнир. После феерического первого четвертьфинала с «Манчестер Сити» (3:3) мало кто ставил на команду Карло Анчелотти. (https://football-pitch.ru/istoriya-evrokubkov/liga-chempionov/final-ligi-chempionov-2024-goda.html)

Дортмундцы выигрывали Лигу чемпионов только один раз, еще в 1997 году, победив в мюнхенском финале «Ювентус».
Футбол Эдин Терзич: от болельщика с «Жёлтой стены» до финала Лиги чемпионов с «Боруссией»Главный тренер дортмундского клуба прошел невероятный путь с командой и надеется огорчить фаворита в финале ЛЧ Вчера в 00:25. (https://www.livesport.ru/football/2024/05/30/champios_league_final/)

Лига чемпионов УЕФА 2024/2025: таблица и календарь, список бомбардиров и статистика игроков, обзоры матчей и видео голов, прогнозы и ставки на матчи... (https://soccer365.ru/competitions/19/)

Сегодня, 1 июня, состоялся финал Лиги чемпионов 2024 года, в котором в Лондоне на стадионе «Уэмбли» встречались дортмундская «Боруссия» и мадридский «Реал». (https://sport24.ru/football/news-652635-kto-vyigral-lch-pobeditel-ligi-chempionov-2024)

Это единственная команда, выигрывавшая ЛЧ пять раз подряд (тогда — Кубок Европейских чемпионов), и единственная команда, выигрывавшая турнир в современном формате три раза подряд.
Клуб выигрывал Лигу чемпионов в 1997 году, в финале того турнира команда одолела итальянский «Ювентус» (3:1). В 2013 году в финале ЛЧ дортмундцы уступили «Баварии» (1:2). (https://www.forbes.ru/sport/513835-real-vyigral-ligu-cempionov-v-15-j-raz)

1 июня 2024 года в финале Лиги чемпионов состоялся матч «Боруссия Дортмунд» ― «Реал Мадрид». Команды встретились на стадионе «Уэмбли» в Лондоне. Игру обслужил судья Славко Винчич.
Эта победа сделала мадридский клуб 15-кратным победителем Лиги чемпионов, на сегодняшний день это рекорд по победам в турнире. (https://rutube.sport/video/borussiya-dortmund-real-madrid-obzor-finala-ligi-chempionov/)

Кто фаворит финала Лиги чемпионов 2024 года: «Реал» или «Боруссия». Где и когда смотреть финал: дата и время. Будет ли трансляция финала на «Матч ТВ». (https://journal.tinkoff.ru/ucl-final-2024-preview/)

Dortmund vs Real Madrid The Road to the Champions League Final 2024. (https://www.youtube.com/watch?v=4y1kzSxIW7Q)

In [14]:
query = "Что такое GigaChain?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Что такое GigaChain?

### Ответ
GigaChain - это фреймворк на основе библиотеки LangChain для создания приложений с применением больших языковых моделей (LLM), таких как GigaChat. Он предоставляет стандартный интерфейс для создания цепочек, различные интеграции с другими инструментами и готовые цепочки для популярных приложений. GigaChain SDK - это библиотека инструментов, которая упрощает и автоматизирует работу с GigaChat, позволяя разработчикам создавать собственные продвинутые ИИ-приложения, использующие технологии Сбера.

### Источники
GigaChain дает доступ к стандартному интерфейсу для создания цепочек, различным интеграциям с другими инструментами и готовым цепочкам для популярных приложений.
GigaChain предоставляет стандартный интерфейс для создания памяти, коллекцию реализаций памяти и примеры цепочек и агентов, которые используют память. (https://developers.sber.ru/docs/ru/gigachain/overview)

GigaChain предоставляет стандартный интерфейс для создания цепочек, различные интеграции с другими инструментами и готовые цепочки для популярных приложений.
GigaChain предоставляет стандартный интерфейс для создания памяти, коллекцию реализаций памяти и примеры цепочек и агентов, которые используют память. (https://github.com/ai-forever/gigachain)

GigaChain SDK — это библиотека инструментов для упрощения и автоматизации работы c GigaChat.
GigaChain — это версия на базе библиотеки LangChain для работы с русским языком, что позволяет использовать её при работе и с другими языковыми моделями. (https://courses.sberuniversity.ru/llm-gigachat/2/4/1)

GigaChain содержит большое количество инструментов и примеров, которые можно использовать в качестве основы своего проекта или просто изучать для расширения кругозора в области возможных кейсов применений LLM. (https://habr.com/ru/companies/sberdevices/articles/794773/)

Gigachain - это фреймворк на основе библиотеки Langchain для создания приложений с применением больших языковых моделей. (https://www.youtube.com/watch?v=eo5LyjRobic)

pip install gigachain or pip install langsmith &amp;&amp; conda install gigachain -c conda-forge. What is this? Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. (https://pypi.org/project/gigachain/)

Кроме того, также представлен GigaChain (SDK) — это адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API. Она позволит разработчикам создавать собственные продвинутые ИИ-приложения, использующие технологии Сбера (https://kod.ru/gigachat-api-gigachain)

краткий пересказ видео от нейросети. GigaChain. (https://300.ya.ru/v_apNGnQpg)

The common name of the sons of Giga Chad, whose number is known only to Giga Chad himself. The Gigachain includes people whose character, behavior, appearance and moral values can compete only with the Giga Chad itself. (https://www.urbandictionary.com/define.php?term=Gigachain)

В этот раз мы подробно поговорим о формировании такого источника из различных файлов. В библиотеках GigaChain/LangChain для этого есть специальные инструменты под названием загрузчики. Мы разберем, как настроить загрузчики и интегрировать их в вашу систему для получения хорошего результата. (https://rutube.ru/video/985ce2480fc21216ceccb7cb43172f95/)

Gigachain is a technology platform using blockchain to protect workers worldwide from exploitation and labour abuse... (https://twitter.com/gigachain_news)

GigaChain - это open-source форк популярной библиотеки LangChain. SDK содержит множество блоков с типовыми решениями по работе с GigaChat, что позволяет разработчику сфокусироваться на более крупных задачах. (https://tenchat.ru/media/2252588-osvoyeniye-otechestvennykh-instrumentov-gigachat-i-gigachain-na-deephackagents)

Эта библиотека позволяет разработчикам создавать собственные продвинутые приложения с использованием искусственного интеллекта. GigaChain стандартизирует типовые сценарии использования различных языковых моделей, таких как работа с текстами, базами данных и документами, упрощая и ускоряя процесс создания ИИ-приложений. (https://www.ferra.ru/news/techlife/neiroset-sbera-stala-dostupna-dlya-razrabotchikov-15-12-2023.htm)

GigaChain is a decentralized financial payment network that rebuilds the traditional payment stack on the blockchain. It utilizes a basket of fiat-pegged stablecoins, algorithmically stabilized by its reserve currency GIGA, to facilitate programmable payments and open financial infrastructure development. (https://thebittimes.com/token-GIGA-BSC-0x482fdac39b7965a7cb31a8601be6a877a764a5de.html)

Приводы серии GIGAchain для рулонных ворот оснащены мощным редуктором, который можно комбинировать с самыми современными электродвигателями. Преимущества изделия. Гибкая система Электронная система концевых выключателей, поставляемая в серийной комплектации, может использоваться в сочетании с интегрированной системой управления при работе с любым имеющимся в продаже трехкнопочным пультом. (https://www.sommer.eu/ru-ru/produkcii/privody-dlja-promyshlennyx-vorot/gigachain)

Получить резюме текста не так уж и сложно: GigaChain + Python • Исследования и разработки В.К. Иванова Суммаризация и реферирование документов... (https://www.ivkconsulting.ru/%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5-%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0-gigachain/)

Copyright©2022. Created with love by Gigachain. (https://gigachian.netlify.app/)

Check GIGACHAIN (GIGACHAIN) price, transactions and holders dynamic chart. Find GIGACHAIN on DEX &amp; check for scam. GIGACHAIN token (coin) running on Ethereum. (https://recentcoin.com/token/gigachain-gigachain-0xdde2c8b2)

On top of that, one unique thing about Gigachains is that it exclusively offers currency trading. That includes both traditional fiat forex trading as well as digital currencies and fiat/crypto pairs. Now while that may deter non-currency traders, it’s by no means an inherently negative thing. (https://www.financebrokerage.com/gigachains-review/)

Overall, Gigachain exudes transparent energy. When you visit the Gigachain website, you get a clear sense of the company. The site is well-designed and simple to navigate, and it provides a detailed overview of the company’s mission and goals. (https://www.europeanbusinessreview.com/gigachains-review-what-you-need-to-know-about-this-broker/)

Рассказ о том, как создать своего агента с помощью GigaChain SDK на базе GigaChat (и других LLM). (https://aiconf.ru/2024/abstracts/12832)

Gigachains is a potential scam. We suggest you keep yourself away from Gigachains. You may see your amount developing step by step, but there is no chance of a return.
Read the Gigachains review and know why we put this broker on the list of scam brokers. (https://medium.com/@ReportScam01/gigachains-review-9d4da8bd647b)

Несколько готовых программ представлены на GitHub в разделе описания фреймворка gigachain. Очень интересная разработка. Мое внимание привлекло приложение gigashell, котле можно встроить в свой терминал, и общаться с ним через него. (https://pikabu.ru/story/gigachat_api_otkryili_dlya_fizicheskikh_lits_besplatno_10832509)

GigaChain is an alt-chain, and therefore might be an Altcoin or a Sidechain. As a sidechain of Bitcoin Core, GigaChain ends up as a highly ironic implementation of the “plexiglass window” concept of Peter Todd’s “Keep Bitcoin Free” video, because it meets his criteria (https://www.truthcoin.info/blog/gigachain/)

Клиентам также доступен GigaChain (SDK) — адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API, которая позволит разработчикам создавать собственные ИИ-приложения, применяющие технологии Сбера. (https://lenta.ru/news/2023/12/15/biznesu/)

«Сбер» также создал GigaChain (SDK) — это адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API. Таким образом разработчики могут сами создавать ИИ-приложения на базе технологий компании. (https://www.sostav.ru/publication/sber-ii-63312.html)

Всем привет! На связи лид разработки SDK GigaChat’a — Константин Крестников. В этой статье я расскажу о том, что такое GigaChain и как в целом SDK позволяет упростить жизнь разработчика LLM, например, научить LLM давать ответы на вопросы по вашим документам или работать в режиме автономного агента. (https://devsday.ru/blog/details/169066)

Собственные AI-агенты на основе больших языковых моделей GigaChain и GigaChat API. Сбер предлагает бизнесу самостоятельно создавать AI-агентов, которые сами ставят себе задачи, пишут код для их решения, анализируют и визуализируют результаты. (https://www.it-world.ru/it-news/market/217554.html)

Чтобы упростить и ускорить работу с моделями, клиентам также доступен GigaChain (SDK) — адаптированная версия библиотеки LangChain для русского языка с поддержкой GigaChat API, которая позволит разработчикам создавать собственные продвинутые ИИ-приложения, применяющие технологии Сбербанка. (https://www.cnews.ru/news/line/2023-12-15_biznesu_i_razrabotchikam)

I need not mention in this GigaChains review how forex trading has taken the trading industry by storm ever since it went online.
GigaChains is a forex trading platform that offers a variety of features and tools to help you trade forex successfully. (https://techbullion.com/gigachains-review-2022-everything-you-need-to-know-about-the-company/)